In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
from lsst.sitcom.vandv.efd import create_efd_client
%matplotlib inline

In [ ]:
# Just using this in the notebook for now.
def merge_packed_PSD(packed_dataframe, base_field, sensor_names):
    """Select fields that represent the Power Spectral Density of \                                                          
    a sensor and unpack them into a dataframe with PSD vs frequency.                                                         
    Parameters                                                                                                               
    ----------                                                                                                               
    packed_dataframe : `pandas.DataFrame`                                                                                    
        packed data frame containing the desired data                                                                        
    base_field :  `str`                                                                                                      
        Base field name that will be expanded to query all                                                                   
        vector entries.                                                                                                      
    sensor_name :  `str` or list                                                                                             
        Name of the sensor(s) of interest.                                                                                   
    Returns                                                                                                                  
    -------                                                                                                                  
    result : `pandas.DataFrame`                                                                                              
        A `pandas.DataFrame` containing the results of the query.                                                            
    """
    minPSDFrequency = packed_dataframe['minPSDFrequency'][0]
    maxPSDFrequency = packed_dataframe['maxPSDFrequency'][0]
    numDataPoints = packed_dataframe['numDataPoints'][0]
    if isinstance(sensor_names, str):
        sensor_names = [sensor_names, ]

    packed_dataframe = packed_dataframe.loc[packed_dataframe.sensorName.isin(sensor_names)]
    packed_fields = [k for k in packed_dataframe.keys()
                     if k.startswith(base_field) and k[len(base_field):].isdigit()]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID                         
    npack = len(packed_fields)
    assert numDataPoints == npack, "Number of packed data points does not match numDataPoints!"
    packed_len = len(packed_dataframe)
    output = np.empty(npack * packed_len)
    deltaF = float(maxPSDFrequency - minPSDFrequency) / (npack - 1) # Frequency step                                         
    columns = []
    for i in range(npack):
        label = f"{base_field}{i}"
        columns.append(minPSDFrequency + i * deltaF)
        output[i::npack] = packed_dataframe[label]
    output = np.reshape(output, (packed_len, npack))
    return pd.DataFrame(data=output, columns=columns, index=packed_dataframe.index)

# This dictionary defines the axis scramble
trueAxes = {'AuxTel-M1':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-M2':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-Truss':{'X':'El', 'Y':'Opt', 'Z':'Az'}}

In [ ]:
client = create_efd_client()
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
psds = []
mount = []

for expId in [2023031400400, 2023031400410, 2023031400414, 2023031400525, 2023031400530, 2023031400540]:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    start = Time(mData['DATE-BEG'])
    end = Time(mData['DATE-END'])
    print(start, end)
    plt.figure(figsize=(8.5, 11))
    plt.subplots_adjust(wspace=0.5, hspace=1.5)
    plt.suptitle(f"Accelerometer / Anemometer / Mount - {expId}", fontsize=16)
    # First, the accelerometer data
    accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)
    indexCounter = 0 # First PSD in the sequence
    M12_axes = ['X', 'Y', 'Z']
    truss_axes = ['X', 'Z', 'Y']
    sensors = ["AuxTel-M1", "AuxTel-M2", "AuxTel-Truss"]
    plotCounter = 1
    psd_sum = []

    for sensor in sensors:
        if sensor in ["AuxTel-M1", "AuxTel-M2"]:
            axes = M12_axes
        else:
            axes = truss_axes
        for axis in axes:
            base_field = f"accelerationPSD{axis}"
            trueAxis = trueAxes[sensor][axis]
            plt.subplot(5,3,plotCounter)
            plt.title(f"{sensor} - {trueAxis}\n", fontsize=12)
            df = merge_packed_PSD(accel_data, base_field, sensor)
            row = df.iloc[indexCounter][2:]
            row.plot()
            sum = row.sum()
            #print(sensor, trueAxis, sum)
            psd_sum.append(sum)

            plt.xlabel('Frequency [Hz]')
            plt.ylabel('PSD [m^2/(Hz s^4)]')
            plt.ylim(0.0, 5.0E-10)
            plotCounter += 1

    psds.append(psd_sum)
    # Next the anemometer data
    # Dat is only once per minute, so need to expand time
    anemom_start = start - TimeDelta(30.0, format='sec')
    anemom_end = start + TimeDelta(30.0, format='sec')

    ane = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['*'],  anemom_start, anemom_end)
    ux_mean = ane['speed0'].values[0]
    ux_std = ane['speedStdDev0'].values[0]
    uy_mean = ane['speed1'].values[0]
    uy_std = ane['speedStdDev1'].values[0]
    uz_mean = ane['speed2'].values[0]
    uz_std = ane['speedStdDev2'].values[0]

    plt.subplot(5,3,plotCounter)
    plt.title("Anemom-UX")
    plt.text(0.1, 0.7,f"Mean={ux_mean:.2f}",horizontalalignment='left',verticalalignment='center')
    plt.text(0.1, 0.3,f"Std={ux_std:.2f}",horizontalalignment='left',verticalalignment='center')
    plotCounter += 1
    plt.subplot(5,3,plotCounter)
    plt.title("Anemom-UY")
    plt.text(0.1, 0.7,f"Mean={uy_mean:.2f}",horizontalalignment='left',verticalalignment='center')
    plt.text(0.1, 0.3,f"Std={uy_std:.2f}",horizontalalignment='left',verticalalignment='center')
    plotCounter += 1
    plt.subplot(5,3,plotCounter)
    plt.title("Anemom-UZ")
    plt.text(0.1, 0.7,f"Mean={uz_mean:.2f}",horizontalalignment='left',verticalalignment='center')
    plt.text(0.1, 0.3,f"Std={uz_std:.2f}",horizontalalignment='left',verticalalignment='center')
    plotCounter += 1

    # Now the mount data
    # Plotting the mount plots
    az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle", start, end)
    el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "elevationCalculatedAngle", start, end)
    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms 
    mount.append([image_az_rms, image_el_rms])
    # Azimuth axis
    plt.subplot(5,3,plotCounter)
    plt.plot(fit_times, az_error, color='red')

    plt.title(f"Azimuth RMS error = {az_rms:.2f} arcseconds\n"
              f"  Image RMS error = {image_az_rms:.2f} arcseconds", fontsize=10)
    plt.ylim(-4.0, 4.0)
    plt.xticks([])
    plt.ylabel("Arcseconds")
    plotCounter += 1

    # Elevation axis
    plt.subplot(5,3,plotCounter+1)
    plt.plot(fit_times, el_error, color='green')
    plt.title(f"Elevation RMS error = {el_rms:.2f} arcseconds\n"
              f"    Image RMS error = {image_el_rms:.2f} arcseconds", fontsize=10)
    plt.ylim(-4.0, 4.0)
    plt.xticks([])



In [ ]:
# Now make the summary plot
mount = np.array(mount)
psds = np.array(psds)
fanSetting = [0.0, 10.0, 20.0, 30.0, 40.0, 50.0]
plt.figure(figsize = (8,8))
plt.subplots_adjust(hspace=0.3)
plt.subplot(2,1,1)
plt.title("M2 Acccelerometer Total Power")
plt.plot(fanSetting, psds[:,3], label="M2-El")
plt.plot(fanSetting, psds[:,4], label="M2-Az")
plt.plot(fanSetting, psds[:,5], label="M2-Opt")
plt.legend()
plt.xlabel("FanSetting(Hz)")
plt.ylabel("Total Power (m^2/s^4)")
plt.subplot(2,1,2)
plt.title("Mount Errors")
plt.plot(fanSetting, mount[:,0], label="Az")
plt.plot(fanSetting, mount[:,1], label="El")
plt.legend()
plt.xlabel("FanSetting(Hz)")
plt.ylabel("RMS Error (arcseconds)")
